## 1.2 Dataset

In [11]:
# !pip install glob2

In [12]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np
import sys

# Configurations principales de nos modèles
IMG_SIZE          = 224             # taille coté final d'une image en pixel (ici 28x28)
NB_EPOCHS_DENOISE = 40               # nombre epoch alogithme debruiter
BATCH_SIZE        = 8            # taille batch de traitement
SAVE_MODEL_DENOISE = "denoiser.h5"     # sauvegarde du modele de debruitage

def process(image):
    image = tf.cast(image/255. ,tf.float32)
    return image

# Import du .env
import dotenv
import os

# Chargement du .env !!!!!!!!!!!! CHANGER LE PATH !!!!!!!!!!!!!!
# Renvoie true si le .env est chargé
dotenv.load_dotenv('/tf/science-de-la-donnee/.env.local')

models_path = os.environ.get('MODELS_PATH_LIVRABLE2')
script_path = os.environ.get('SCRIPT_PATH_LIVRABLE2')
sys.path.insert(0, models_path)
sys.path.insert(0, script_path)

import builder_vae
import homemade
import test2

SAVE_WEIGHTS_PATH = os.environ.get('WEIGHT_PATH_LIVRABLE2')
SOURCE_LIVRABLE2_PATH = os.getenv("SOURCE_LIVRABLE2_PATH")

print(SOURCE_LIVRABLE2_PATH)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/tf/data/livrable2


In [13]:

x_train, x_test = image_dataset_from_directory(
    SOURCE_LIVRABLE2_PATH,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    # label_mode=None,
    shuffle=False,
    validation_split=0.2,
    subset="both",
    seed=123,
    color_mode="rgb",
)

AUTOTUNE = tf.data.experimental.AUTOTUNE

x_train = x_train.map(lambda x,y: (x/255,y))
x_test = x_test.map(lambda x,y: (x/255,y))

KeyboardInterrupt: 

In [ ]:
%ls ./photoOnly/Photo

ls: cannot access './photoOnly/Photo': No such file or directory


Commençons par écrire une fonction qui permet de visualiser $n$ premiers enregistrements en noir et blanc. 

In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
# os.chdir(r'/tf')
from keras_cv.layers import RandomGaussianBlur
from random import randint, seed

def add_noise(img, noise_factor=25, blur_factor=5):
    r = tf.experimental.numpy.random.randint(1, 4)
    noised_img = img
    # if r == 1:
    #     noised_img = RandomGaussianBlur(kernel_size=3, factor=(0.5, 2))(img)
    # elif r == 2:
    #     noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_factor/255, dtype=tf.float32)
    #     noised_img = tf.cast(img, tf.float32) + noise
    # elif r == 3:
    noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_factor/255, dtype=tf.float32)
    noised_img = tf.cast(img, tf.float32) + noise
    noised_img = RandomGaussianBlur(kernel_size=5, factor=(0.5, blur_factor))(noised_img)
    return noised_img, img

#  divide by 255


x_train_noisy = x_train.map(lambda x,y: (add_noise(x, 15, 1.5)))
x_test_noisy = x_test.map(lambda x,y: (add_noise(x, 15, 1.5)))


def display_images(X, n):
    plt.figure(figsize=(7, 7))
    # print(X.shape)
    for i in range(n):
    #A COMPLETER
        # print(X[i].shape)
        ax = plt.subplot(1, n, i + 1)
                         #A COMPLETER
        plt.imshow(X[i])
                   #A COMPLETER
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

ModuleNotFoundError: No module named 'keras_cv'

In [ ]:
display_images(list(x_train_noisy.take(1).as_numpy_iterator())[0][0], n=2)
display_images(list(x_train.take(1).as_numpy_iterator())[0][0], n=2)

: 

: 

: 

Voyons ce que ça donne :

# AutoEncoder

In [ ]:
MODEL_CHOSEN = "test2"

def load_model(model_choosen):
    match(model_choosen):
        case 'homemade':
            model = homemade.build(IMG_SIZE)
        case 'vae':
            model = builder_vae.build(IMG_SIZE, 64)
        case 'test2':
            model = test2.build(IMG_SIZE)
    return model
autoencoder = load_model(MODEL_CHOSEN)
autoencoder.summary()

: 

: 

: 

###### Entrainement de l'auto-encodeur

On va ensuite entraîner l'auto-encodeur en utilisant les constantes définit au début (`NB_EPOCHS_DENOISE,BATCH_SIZE`)

In [ ]:
# path = createTrainingData.create_training_data(weight_path, model, model_choosen, num_classes, image_h, image_w, batch_size)

# checkpoint_path = path+"/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# weights_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path,
#     verbose=1,
#     save_weights_only=True,
#     save_freq='epoch')

# history = model.fit(train_set, epochs=epochs, validation_data=test_set, callbacks=[weights_callback])


history = autoencoder.fit(
    x_train_noisy,
    epochs=1,
    shuffle=True,
    validation_data=x_test_noisy
)

: 

: 

: 

Affichez maintenant la courbe d'apprentissage. Que pensez-vous des performances du modèle ?

In [ ]:
model_weight_path = SAVE_WEIGHTS_PATH + 'TO_\CHANGE/weights.h5' 
autoencoder.save_weights(model_weight_path)

: 

: 

: 

In [ ]:
pred = autoencoder.predict(x_test_noisy)    

: 

: 

: 

In [ ]:
display_images(list(x_test_noisy.take(1).as_numpy_iterator())[0][0], n=1)
display_images(pred, n=1)

: 

: 

: 

In [ ]:
# Visualisation des pertes d'apprentissage (Train) et de validation (Test)
plt.plot( #A COMPLETER
         label='train')
plt.plot( #A COMPLETER
         label='test')
plt.legend()

: 

: 

: 

Que pensez-vous des performances du modèle ?
<em>À COMPLÉTER</em>


# 1.4 Sauvgarde de l'auto-encodeur

L'entrainement de l'auto-encodeur sans utilisation de puissance de calcul (GPU) peut prendre beaucoup de temps. Usuellement, nous sauvegardons le modèle entraîné en local ou sur un serveur distant pour l'utiliser ultérieurement afin de traiter les nouvelles données (d'ailleurs, vous verrez une utilisation avancée de cette technique, le transfert learning, dès la semaine prochaine).
Pour sauvegarder le modèle `autoencoder`, utiliser la méthode `save`.

In [ ]:
tf.keras.backend.clear_session()


: 

: 

: 

In [ ]:
decoded_imgs = #A COMPLETER


: 

: 

: 

: 

: 

: 

: 

: 

: 